# Part 1: Introduction to Umami and the `Metric` Class

Umami is a package for calculating metrics for use with for Earth surface dynamics models. This notebook is the first half of a two-part introduction to using umami.

Umami was designed to work well with the [terrainbento](https://terrainbento.readthedocs.io/en/latest/) model package, as well as other models built using the [Landlab Toolkit](https://github.com/landlab/landlab). 

## Scope of this tutorial

In this tutorial you will learn the basic principles behind the umami package and use the `Metric` class to calculate terrain statistics. 

If you have comments or questions about the notebooks, the best place to get help is through [GitHub Issues](https://github.com/TerrainBento/umami/issues).

To begin this example, we will import the required python packages. 

In [1]:
import warnings
warnings.filterwarnings('ignore')

from io import StringIO
from landlab import RasterModelGrid, imshow_grid
from umami import Metric
from umami.calculations import aggregate

ModuleNotFoundError: No module named 'umami'

## Step 1: Create an example grid
First we will create an example grid and add a field called `topographic__elevation` to it. The grid shown here has a shape of (10x10) and slopes to the south-west.

The Landlab grid is used as the core data structure for umami as it provides important information about grid size, shape, adjacency. It, however, is very flexible as it is compatible with regular and irregular grids and can be specified with a numpy array. 

In [ ]:
grid = RasterModelGrid((10, 10))
z = grid.add_zeros("node", "topographic__elevation")
z += grid.x_of_node + grid.y_of_node

imshow_grid(grid, z)

## Step 2: Use the `umami.Metric` class

Umami was designed to facilitate the calculation of topographic metrics for use in model analysis. For example, say one wanted to assess a model's performance based on the mean topography. On one hand, this can simply be done by evaluating `np.mean(z)` and writing the results out to an output file. 

However, if the set of metrics becomes more complex, then extensive analysis must occur in the driver script or function used to run the model and produce model analysis output. If one wanted to take multiple aggreggation statistics on a single field (e.g., `topographic__elevation`) and to also calculate other metrics, then using the umami package becomes competitive relative to hand-calculating a few metrics. 

To use the umami [`Metric`](https://umami.readthedocs.io/en/latest/umami.metric.html) class we need to pass a grid, such as that defined above, as well as a dictionary indicating what metrics we want calculated. Metrics can be added after instantiation, and can also be added using an input file in [YAML](https://yaml.org) format. 

Each calculation gets its own unique name (the key in the dictionary), and is associated with a value, a dictionary specifying exactly what should be calculated. The only value of the dictionary required by all umami calculations is `_func`, which indicates which of the [`umami.calculations`](https://umami.readthedocs.io/en/latest/umami.calculations.html) will be performed. Subsequent elements of this dictionary are the required inputs to the calculation function and are described in their documentation. 

Note that some calculations listed in the [`umami.calculations`](https://umami.readthedocs.io/en/latest/umami.calculations.html) submodule are valid for both the umami `Metric` and `Residual` classes, while others are for `Residual`s only (the `Residual` will be covered in [Part 2](IntroductionToResidual.ipynb) of this notebook series). 

The order that calculations are listed is read in as an [OrderedDict](https://docs.python.org/3/library/collections.html#collections.OrderedDict) and retained as the "calculation order". 

In our example we will use the following dictionary: 

```python
metrics = {
    "me": {
        "_func": "aggregate",
        "method": "mean",
        "field": "topographic__elevation"
    },
    "ep10": {
        "_func": "aggregate",
        "method": "percentile",
        "field": "topographic__elevation",
        "q": 10
    }
}
```
This specifies calculation of the mean of `topographic__elevation` (to be called "me") and the 10th percentile `topographic__elevation` (called "ep10"). The equivalent portion of a YAML input file would look like:

```yaml
metrics:
    me:
        _func: aggregate
        method: mean
        field: topographic__elevation
    ep10:
        _func: aggregate
        method: percentile
        field: topographic__elevation
        q: 10
```

Next, let's construct the Metric class. It is important to note that, the umami [`aggregate`](https://umami.readthedocs.io/en/latest/umami.calculations.metric.aggregate.html) function only operates on the core nodes of the model grid. Use the link above to read the API documentation that describes in detail what values the `aggreggate` calculation expects. 

In [ ]:
metrics = {
    "me": {
        "_func": "aggregate",
        "method": "mean",
        "field": "topographic__elevation"
    },
    "ep10": {
        "_func": "aggregate",
        "method": "percentile",
        "field": "topographic__elevation",
        "q": 10
    }
}

metric = Metric(grid, metrics=metrics)

The umami `Metric` does not calculate the metrics when it is instantiated. During instantiation, umami performs many checks, for example, ensuring that all needed fields are present on the grid. To calculate the metrics, run the `calculate` bound method. 

In [ ]:
metric.calculate()

Now that the metrics have been calculated we highlight the most useful attributes of the `Metric class`.

`metric.names` gives the names of the metrics as a list, in calculation order. 

In [ ]:
metric.names

`metric.values` gives the values of the metrics as a list, in calculation order. 

In [ ]:
metric.values

A function is available to get the value of a given metric, given a name.

In [ ]:
metric.value("me")

## Step 3: Use umami calculations without the `Metric` class

Each of the umami calculation functions (for example `aggregate`) can be used as a stand-alone function without the `Metric` class. For example, the following is how one would use it to perform the two calculations done above:

In [ ]:
aggregate(grid, method="mean", field="topographic__elevation")

In [ ]:
aggregate(grid, method="percentile", field="topographic__elevation", q=10)

The API documentation for each calculation shows its use as part of a `Metric` or `Residual`, and as a stand-alone function.

## Step 4: Add new metrics

Next we will highlight the ability to add new metrics to the umami `Metric` by adding the variance and the 30th percentile. 

In [ ]:
new_metrics = {
    "va": {
        "_func": "aggregate",
        "method": "var",
        "field": "topographic__elevation"
    },
    "ep30": {
        "_func": "aggregate",
        "method": "percentile",
        "field": "topographic__elevation",
        "q": 30
    }
}

metric.add_from_dict(new_metrics)
metric.calculate()

First we examine the names, 

In [ ]:
metric.names

And then the values. 

In [ ]:
metric.values

## Step 5: Write output

Umami was designed to interface well with model analysis tools like [`scipy.optimize`](https://docs.scipy.org/doc/scipy/reference/optimize.html) or [Dakota](http://dakota.sandia.gov). To this end, in addition to the output provided in `metric.names` and `metric.values`, two additional methods for writing output are built in:

In [ ]:
out = StringIO()
metric.write_metrics_to_file(out, style="dakota")
file_contents = out.getvalue().splitlines()
for line in file_contents:
    print(line.strip())

In [ ]:
out = StringIO()
metric.write_metrics_to_file(out, style="yaml")
file_contents = out.getvalue().splitlines()
for line in file_contents:
    print(line.strip())

# Next steps

Now that you have a sense for how the `Metric` class is used, try the next notebook: [Part 2: Introduction to Umami and the `Residual` Class](IntroductionToResidual.ipynb).